<CENTER>
    <a href="http://opendata.atlas.cern" class="icons"><img src="http://opendata.atlas.cern/DataAndTools/pictures/opendata-top-transblack.png" style="width:40%"></a>
</CENTER>

# A more difficult notebook in python

### In this notebook you can find a more difficult program that shows further high energy physics (HEP) analysis techniques.

##### The following analysis is searching for events where [Z bosons](https://en.wikipedia.org/wiki/W_and_Z_bosons) decay to two leptons of same flavour and opposite charge (to be seen for example in the [Feynman diagram](https://en.wikipedia.org/wiki/Feynman_diagram)).

<CENTER><img src="images/Z_ElectronPositron.png" style="width:40%"></CENTER>

First of all - like we did it in the first notebook - ROOT is imported to read the files in the _.root_ data format.

In [ ]:
import ROOT
import math

Next we have to open the data that we want to analyze. As described above the data is stored in a _*.root_ file.

In [ ]:
f = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_105986.ZZ.root")

After the data is opened we create a canvas on which we can draw a histogram. If we do not have a canvas we cannot see our histogram at the end. Its name is _Canvas_ and its header is _c_. The two following arguments define the width and the height of the canvas.

In [ ]:
canvas = ROOT.TCanvas("Canvas","c",800,600)

The next step is to define a tree named _t_ to get the data out of the _.root_ file.

In [ ]:
tree = f.Get("mini")

Now we define a histogram that will later be placed on this canvas. Its name is _variable_, the header of the histogram is _Mass of the Z boson_, the x axis is named _mass [GeV]_ and the y axis is named _events_. The three following arguments indicate that this histogram contains 30 bins which have a range from 60 to 120.

In [ ]:
hist = ROOT.TH1F("variable","Mass of the Z boson; mass [GeV]; events",30,60,120)

Time to fill our above defined histogram. At first we define some variables and then we loop over the data. We also make some cuts as you can see in the # _comments_.

In [ ]:
for event in tree:
    
    # Cut #1: At least 2 leptons
    if tree.lep_n == 2:
        
        # Cut #2: Leptons with opposite charge
        if (tree.lep_charge[0] != tree.lep_charge[1]):
            
            # Cut #3: Leptons of the same family (2 electrons or 2 muons)
            if (tree.lep_type[0] == tree.lep_type[1]):
                
                # Let's define the calculations for invariant mass
                theta_0 = 2*math.atan(math.exp(-tree.lep_eta[0]))
                theta_1 = 2*math.atan(math.exp(-tree.lep_eta[1]))
                p_0 = tree.lep_pt[0]/math.sin(theta_0)
                p_1 = tree.lep_pt[1]/math.sin(theta_1)
                pz_0 = p_0*math.cos(theta_0)
                pz_1 = p_1*math.cos(theta_1)
                px_0 = p_0*math.sin(theta_0)*math.cos(tree.lep_phi[0])
                px_1 = p_1*math.sin(theta_1)*math.cos(tree.lep_phi[1])
                py_0 = p_0*math.sin(theta_0)*math.sin(tree.lep_phi[0])
                py_1 = p_1*math.sin(theta_1)*math.sin(tree.lep_phi[1])
                sumpz = pz_0 + pz_1
                sumpx = px_0 + px_1
                sumpy = py_0 + py_1
                sumE = tree.lep_E[0] + tree.lep_E[1]
                invMass = math.sqrt(sumE**2 - sumpz**2 - sumpx**2 - sumpy**2)/1000
                # divide by 1000 to get in GeV units
                
                hist.Fill(invMass)

After filling the histogram we want to see the results of the analysis. First we draw the histogram on the canvas and then the canvas on which the histogram lies.

In [ ]:
hist.Draw()

In [ ]:
canvas.Draw()